In [1]:
!pip install keras_applications
!pip install transformers

     |████████████████████████████████| 50 kB 261 kB/s            


In [2]:
import numpy as np 
import pandas as pd 
import os
import re
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing import image
import time
from nltk.corpus import wordnet as wn
from PIL import Image
import requests
import torch
from transformers import DeiTFeatureExtractor, DeiTForImageClassification
from transformers import AutoFeatureExtractor, ViTForImageClassification

import cv2
import matplotlib.pyplot as plt

In [3]:
in_path = '../input/dataset-crawler-hyponyms/'
data_dict = {}
for path, dirs, files in os.walk(in_path):
    for i in dirs:
        for path2, dirs2, files2 in os.walk('../input/dataset-crawler-hyponyms/'+i):
            urls = [in_path+i+'/'+j for j in files2]
            data_dict[str(i)] = urls
print(len(data_dict))

10


In [4]:
data_dict.keys()

dict_keys(['tabby cat', 'angora cat', 'lynx cat', 'siamese cat', 'tiger cat', 'persian cat', 'cougar cat', 'leopard cat', 'egyptian cat', 'cat'])

# Classifiers

In [5]:
def evaluate(pred_class, actual_class, acc, TP, FN, sp_list, lch_list, wups_list, siblings, common_misconceptions):
    # find wordnet synsets for predicted class 
    wn_preds = wn.synsets(pred_class, pos=wn.NOUN)
    wn_pred=wn_preds[0]
    
    # find synset of ground truth class
    try:            
        actual_class=actual_class.replace(' ', '_')
        wn_label = wn.synsets(actual_class, pos=wn.NOUN)[0]
    except:   # we have added the word 'cat' for disambiguation (e.g. tiger cat vs tiger)
        actual_class = actual_class.replace('_cat', '')
        wn_label = wn.synsets(actual_class, pos=wn.NOUN)[0]
    #print('Actual:', wn_label, actual_class)
    #print('Pred:', wn_preds, pred_class)
    
    # Find hypernyms of ground truth class and predicted class 
    try:
        label_hyper_actual = wn_label.hypernyms()[0]
    except: # if there are no hypernyms, get entity as the global hypernym
        label_hyper_actual = wn.synsets('entity')[0]
    try:
        label_hyper_pred = wn_pred.hypernyms()[0]
    except:
        label_hyper_pred = wn.synsets('entity')[0]
    #print('Hypernyms:', label_hyper_pred, label_hyper_actual)
    
    # check if predicted class is a sub-class of actual label 
    # first, find all hyponyms (in any taxonomy depth) of the ground truth class
    label_hypo = list(set([w for s in wn_label.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    
    # then, find all hyponym synsets
    synset_hypo = [wn.synsets(word, pos=wn.NOUN)[0] for word in label_hypo]
    
    # if predicted is subset, then prediction is correct
    hypo_intersection=set(wn_preds).intersection(set(synset_hypo))
    same=False
    
    # if there is an overlap in actual/pred synsets or label names
    if hypo_intersection or (wn_label in wn_preds) or (actual_class.lower()==pred_class.lower()):
        #print('Same')
        acc+=1
        TP+=1
        same=True
        
    # if actual/pred synsets have a common immediate parent node (or entity as the common parent)
    elif (label_hyper_pred==label_hyper_actual) and not(label_hyper_pred==wn.synset('entity.n.01') and label_hyper_actual==wn.synset('entity.n.01')):
        #print('Siblings. Hypernyms:', label_hyper_pred)
        FN+=1
        siblings+=1
        common_misconceptions.append((actual_class, pred_class))
        
    # otherwise, ground truth and pred classes have more than 1-level difference
    else:
        #print('Not immediately related')
        FN+=1
        common_misconceptions.append((actual_class, pred_class))
     
    if not same:   # find WordNet-based metrics for different ground truth/pred synsets
        min_hypernym = wn_pred.lowest_common_hypernyms(wn_label)
        #print('Lowest common hypernym: ', min_hypernym)
    
        shortest_path = wn_pred.path_similarity(wn_label)
        sp_list.append(shortest_path)
        #print('Shortest path similarity between {} and {}: {}'.format(pred_class, actual_class, shortest_path))
    
        # hortest path and the maximum depth of the taxonomy
        lch = wn_pred.lch_similarity(wn_label)
        lch_list.append(lch)
        #print('Leacock-Chodorow similarity between {} and {}: {}'.format(pred_class, actual_class, lch))
    
        # depth of the two synsets in the taxonomy and that of their Least Common Subsumer (most specific ancestor node)
        wups = wn_pred.wup_similarity(wn_label)
        wups_list.append(wups)
        #print('Wu-Palmer similarity between {} and {}: {}'.format(pred_class, actual_class, wups))
    
    return acc, TP, FN, sp_list, lch_list, wups_list, siblings, common_misconceptions

In [6]:
def predictions(image, model, feature_extractor, all_time, preds):
    start = time.time()    
    inputs = feature_extractor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    elapsed_time = time.time()-start
    probs = logits.softmax(dim=1)
    idx = torch.topk(probs, 3, largest=True, sorted=True, out=None)
    top3pos = idx[1][0]
    pred_classes = {}
    top_counter = 1
    for item in top3pos:
        predicted_class_idx = item.item()
        pred_classes[top_counter] = model.config.id2label[predicted_class_idx]
        top_counter+=1
    
    # model predicts one of the 1000 ImageNet classes
    preds.append(pred_classes)
    print(pred_classes)
    all_time.append(elapsed_time)
    
    return all_time, preds, pred_classes

In [7]:
# https://huggingface.co/facebook/deit-small-patch16-224

In [8]:
feature_extractor = AutoFeatureExtractor.from_pretrained('facebook/deit-small-patch16-224')
model = ViTForImageClassification.from_pretrained('facebook/deit-small-patch16-224')
model_name = 'DeiT'

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.2M [00:00<?, ?B/s]

In [9]:
results={}
all_misconceptions = {}
for data_name, data in data_dict.items():
    all_time = []
    preds = []
    acc=0
    TP=0
    FN=0
    siblings=0
    common_misconceptions = []
    sp_list, lch_list, wups_list = [], [], []

    for url in data:
        image = cv2.imread(url)
        #try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        all_time, preds, top = predictions(image, model, feature_extractor, all_time, preds)
        top_1_class=list(top.values())[0]
        if ',' in top_1_class:
            splt = top_1_class.split(',')
            top_1_class = splt[0]
            top_1_class=top_1_class.rstrip().lstrip()
        if ' ' in top_1_class:
            top_1_class=top_1_class.replace(' ', '_')
        acc, TP, FN, sp_list, lch_list, wups_list, siblings, common_misconceptions = evaluate(top_1_class, data_name, acc, 
                                                             TP, FN, sp_list, lch_list, wups_list, siblings, common_misconceptions)

        #except Exception as e:
        #    print(str(e))
        mean_acc = acc/(TP+FN)
        #print(mean_acc, TP, FN, np.mean(sp_list), np.mean(lch_list), np.mean(wups_list), np.sum(all_time), siblings)
        run_id = data_name+'_'+model_name
        results[run_id] = (data_name, model_name, mean_acc, TP, FN, np.mean(sp_list), 
                               np.mean(lch_list), np.mean(wups_list), np.sum(all_time), siblings)
        all_misconceptions[run_id] = common_misconceptions

{1: 'tabby, tabby cat', 2: 'Egyptian cat', 3: 'tiger cat'}


/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{1: 'tiger cat', 2: 'tabby, tabby cat', 3: 'Egyptian cat'}
{1: 'tabby, tabby cat', 2: 'Egyptian cat', 3: 'tiger cat'}
{1: 'feather boa, boa', 2: 'Egyptian cat', 3: 'tabby, tabby cat'}
{1: 'tabby, tabby cat', 2: 'tiger cat', 3: 'Egyptian cat'}
{1: 'tabby, tabby cat', 2: 'tiger cat', 3: 'Egyptian cat'}
{1: 'tabby, tabby cat', 2: 'Egyptian cat', 3: 'tiger cat'}
{1: 'tabby, tabby cat', 2: 'Egyptian cat', 3: 'tiger cat'}
{1: 'web site, website, internet site, site', 2: 'tabby, tabby cat', 3: 'menu'}
{1: 'tiger cat', 2: 'tabby, tabby cat', 3: 'Egyptian cat'}
{1: 'Egyptian cat', 2: 'tabby, tabby cat', 3: 'tiger cat'}
{1: 'tiger cat', 2: 'tabby, tabby cat', 3: 'Egyptian cat'}
{1: 'tabby, tabby cat', 2: 'tiger cat', 3: 'Egyptian cat'}
{1: 'tabby, tabby cat', 2: 'tiger cat', 3: 'Egyptian cat'}
{1: "spider web, spider's web", 2: 'tiger cat', 3: 'tabby, tabby cat'}
{1: 'Egyptian cat', 2: 'tabby, tabby cat', 3: 'tiger cat'}
{1: 'Egyptian cat', 2: 'tiger cat', 3: 'tabby, tabby cat'}
{1: 'web site, w

In [10]:
all_df = pd.DataFrame(results.values(), columns=['dataset', 'model', 'accuracy', 'TP', 'FN', 'path_similarity', 'LCH', 'WUPS', 'predict_time', 'siblings'])
all_df

,dataset,model,accuracy,TP,FN,path_similarity,LCH,WUPS,predict_time,siblings
0,tabby cat,DeiT,0.540000,27,23,0.223969,2.012390,0.770161,8.705569,9
1,angora cat,DeiT,0.000000,0,50,0.306284,2.398883,0.841817,7.853393,44
2,lynx cat,DeiT,0.900000,45,5,0.047619,0.593064,0.090909,7.957224,0
3,siamese cat,DeiT,0.900000,45,5,0.161905,1.371428,0.404545,7.680907,2
4,tiger cat,DeiT,0.150000,9,51,0.132330,1.431033,0.615065,8.984464,0
5,persian cat,DeiT,0.920000,46,4,0.203755,1.787278,0.632738,7.669885,2
6,cougar cat,DeiT,0.940000,47,3,0.050961,0.659786,0.300366,7.851832,0
7,leopard cat,DeiT,0.000000,0,50,0.156537,1.487116,0.600985,7.524933,7
8,egyptian cat,DeiT,0.460000,23,27,0.105814,1.311639,0.517496,7.681827,0
9,cat,DeiT,0.925926,25,2,0.059028,0.806106,0.333913,5.116098,0


In [11]:
all_df.to_pickle('./cats_imagenet_DeiT.pkl')

In [12]:
import pickle 

with open('./DeiT_all_common_misconceptions.pkl', 'wb') as f:
    pickle.dump(all_misconceptions, f)
        
with open('./DeiT_all_common_misconceptions.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)